In [1]:
import matplotlib.pyplot as plt 
import torch 
import torch.nn as nn
from torch.utils.data import dataloader
from torchvision import transforms
from torchvision.transforms import ToTensor
from dataloader import DeepGlobe_Road

In [2]:
dataset_dir = "/dataset/deepglobe-2018-road-extraction"

transform = transforms.Compose([
    ToTensor() , 
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) , 
    transforms.PILToTensor() , 
])

In [4]:
train = DeepGlobe_Road(parent_dir = "dataset" , set = "valid" , transform= transform)
len(train)

621